In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,40931
2,Huelva,Confirmados PDIA 14 días,3185
3,Huelva,Tasa PDIA 14 días,"620,6520256445233"
4,Huelva,Confirmados PDIA 7 días,1396
5,Huelva,Tasa PDIA 7 dias,"272,0346084143656"
6,Huelva,Total Confirmados,41143
7,Huelva,Curados,35015
8,Huelva,Fallecidos,396


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  40931.0


/tmp/ipykernel_3270/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11406.0


/tmp/ipykernel_3270/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3270/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3270/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3270/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 422 personas en los últimos 7 días 

Un positivo PCR cada 166 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,40931.0,1396.0,3185.0,513170.0,272.034608,620.652026,182.0
Huelva-Costa,24016.0,775.0,1845.0,289548.0,267.658557,637.200050,107.0
Huelva (capital),11406.0,341.0,866.0,143837.0,237.073910,602.070399,65.0
Condado-Campiña,12878.0,490.0,1014.0,156231.0,313.638138,649.038923,47.0
Sierra de Huelva-Andévalo Central,3651.0,124.0,315.0,67391.0,184.000831,467.421466,24.0
Ayamonte,1751.0,81.0,150.0,21104.0,383.813495,710.765732,12.0
Isla Cristina,2762.0,74.0,193.0,21393.0,345.907540,902.164259,9.0
Valverde del Camino,785.0,17.0,37.0,12750.0,133.333333,290.196078,7.0
Moguer,1780.0,77.0,173.0,21867.0,352.128779,791.146476,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Sanlúcar de Guadiana,25.0,15.0,18.0,403.0,3722.084367,4466.501241,2.0
Cumbres Mayores,118.0,10.0,43.0,1749.0,571.755289,2458.547742,1.0
Corteconcepción,39.0,5.0,13.0,536.0,932.835821,2425.373134,1.0
Marines (Los),14.0,7.0,8.0,399.0,1754.385965,2005.012531,0.0
Nava (La),6.0,3.0,4.0,258.0,1162.790698,1550.387597,0.0
Cañaveral de León,13.0,2.0,6.0,396.0,505.050505,1515.151515,0.0
Escacena del Campo,194.0,22.0,33.0,2287.0,961.958898,1442.938347,0.0
Trigueros,470.0,35.0,89.0,7862.0,445.179344,1132.027474,2.0
Villablanca,243.0,16.0,32.0,2885.0,554.592721,1109.185442,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Encinasola,160.0,1.0,12.0,1310.0,76.335878,916.030534,0.0,0.083333
Jabugo,83.0,1.0,7.0,2260.0,44.247788,309.734513,0.0,0.142857
Santa Ana la Real,21.0,1.0,5.0,475.0,210.526316,1052.631579,0.0,0.200000
Cortegana,266.0,5.0,22.0,4602.0,108.648414,478.053020,0.0,0.227273
Cumbres Mayores,118.0,10.0,43.0,1749.0,571.755289,2458.547742,1.0,0.232558
Puebla de Guzmán,163.0,2.0,8.0,3092.0,64.683053,258.732212,0.0,0.250000
Cerro de Andévalo (El),185.0,3.0,11.0,2327.0,128.921358,472.711646,1.0,0.272727
Punta Umbría,1158.0,42.0,140.0,15355.0,273.526539,911.755129,4.0,0.300000
Villanueva de los Castillejos,189.0,4.0,13.0,2825.0,141.592920,460.176991,1.0,0.307692
